# 卷积运算
输入 * 核 = 输出
核：过滤器，卷积核窗口，卷积窗口

卷积运算与互相关运算类似。

为了得到卷积运算的输出，我们只需要将数组左右翻转并上下翻转，再与输入数组做互相关运算。所以，其结果往往与互相关运算不同。

但是，学出卷积核才是重点，在深度学习中核数组都是学出来的:卷积层无论使⽤互相关运算或卷积运算都不影响模型预测时的输出。

In [1]:
import torch
import torch.nn as nn

In [2]:
def corr2d(X, K):
    # 卷积核大小
    h, w = K.shape
    # 卷积结果
    Y = torch.zeros((X.shape[0] - h + 1, X.shape[1] - w + 1))
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            Y[i, j] = (X[i: i + h, j : j + w] * K).sum()
    return Y

In [4]:
X = torch.tensor([[0, 1, 2], [3, 4, 5], [6, 7, 8]])
K = torch.tensor([[0, 1], [2, 3]])
corr2d(X, K)

tensor([[19., 25.],
        [37., 43.]])

# 二维卷积层
卷积层的模型参数包括了卷积核核标量偏差。

我们先对卷积核随机初始化，然后不断迭代卷积核与偏差

In [5]:
class Conv2D(nn.Module):
    def __init__(self, kernel_size):
        super(Conv2D, self).__init__()
        self.weight = nn.Parameter(torch.randn(kernel_size))
        self.bias = nn.Parameter(torch.randn(1))
        
    def forward(self, x):
        return corr2d(x, self.weight) + self.bias

In [6]:
# 构造 6 * 8的图像
X = torch.ones(6, 8)
X[:, 2:6] = 0
X

tensor([[1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.]])

In [7]:
# 构造卷积核
K = torch.tensor([[1, -1]])

In [9]:
Y = corr2d(X, K)
Y

tensor([[ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.]])

# 通过数据学习核数组
1. 构造卷积层
2. 误差函数估计卷积层输出
3. 计算梯度更新权重

In [11]:
conv2d = Conv2D(kernel_size=(1, 2))
step = 20
lr = 0.01
for i in range(step):
    Y_hat = conv2d(X)
    l = ((Y_hat - Y)**2).sum()
    l.backward()
    
    # 梯度下降
    conv2d.weight.data -= lr * conv2d.weight.grad
    conv2d.bias.data -= lr * conv2d.bias.grad
    # 梯度清零
    conv2d.weight.grad.fill_(0)
    conv2d.bias.grad.fill_(0)
    
    if (i + 1) % 5 == 0:
        print('Step %d, loss %.3f' % (i + 1, l.item()))

Step 5, loss 2.894
Step 10, loss 0.446
Step 15, loss 0.084
Step 20, loss 0.019


In [12]:
print('weight:', conv2d.weight.data)
print('bias:', conv2d.bias.data)

weight: tensor([[ 0.9602, -0.9757]])
bias: tensor([0.0087])


# 特征图和感受野
经过卷积后的输出，可以看作输入在空间维度上某一级的表征，也叫特征图（feature map）。

影响元素x的FP的所有可能输入区域（可能大于输入尺寸）叫做x的感受野（receptive field）。

X: 3 * 3 -卷积-> Y: 2 * 2 -卷积-> Z: 1 * 1

那么Z在Y上的感受野包括Y上的4个元素，Z在X上的感受野包括X上的9个元素